In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from statistics import NormalDist
from scipy.stats import norm
import scipy.special as sp

# Downloading the data:

In [5]:
X = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\Mod-13-BO-Capstone-Session\Mod-13-BO-Capstone-Session\initial_data\function_5\initial_inputs.npy")
y = np.load(r"C:\Users\mo13\OneDrive\Documents\ML course\project\Mod-13-BO-Capstone-Session\Mod-13-BO-Capstone-Session\initial_data\function_5\initial_outputs.npy")

Here I'm appending the inputs resulted from the acquisition function below, so everytime I get an query, I add it here to the input "X"

In [6]:
X = np.append(X,[[0, 0.693878, 0.999999, 0.999999], [0, 0, 0, 0], [0, 0, 0, 0], 
                [0.999999, 0.999999, 0.999999, 0.999999]], axis=0)


Here I'm appending outputs I get from the feedback, so evertime I get the output from the feedback, I add it to the output "y"

In [7]:
y = np.append(y,[2162.457311, 4647.76325, 163.1225, 8662.405001], axis=0)

# Fitting in Guassian Distribution

In [8]:
gpr = GaussianProcessRegressor(kernel=None)
gpr.fit(X, y);

# Creating a grid for grid search

In [9]:
x1 = np.linspace(0, 1, 30)

In [10]:
import itertools as it
dim = 4
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)


Implementing the grid search:

In [11]:
X_grid = np.array(X_grid)
mean, std = gpr.predict(X_grid, return_std = True)

#####################################################################################################
#Below is my previous approach where I used the probability improvement method for the Acquisition Function (you can ignor it)
#ucb = mean + 1.96 * std
#acquisition_function = []
#eta=0.1
#mx = max([2162.457311, 4647])
#for k in range(len(X_grid)):
 #   acquisition_function.append(1 - NormalDist(mu = mean[k], sigma = std[k]).cdf(mx + eta))
  #  if  mean[k] > mx:
   #     mx = mean[k]
#acquisition_function = np.array(acquisition_function)

# Applying the "Analytic LogEI"

I have found this approach in "Unexpected Improvements to Expected Improvement for Bayesian Optimization" article (expalined in README file)

Now let's define log1mexp which is $$log( 1 - exp(x) )$$

In [12]:
def log1mexp(x):
    return np.log(1-np.exp(x))

$log( 1 - exp(x) )$ is used in the acquistion function later

Recall from README file: 

$$c1 = log(2π) / 2$$
$$c2 =  log(π/2) /2$$
$$erfcx(x) = exp(x^{2})erfc(x)$$
where erfc is the complementary error function.
The "Analytic LogEI" acquistion function is: $$ LogEIy^{∗}(x) = log_h((µ(x) − y^{∗})/σ(x)) + log(σ(x))$$
where $log_h(x)$ is calculated below (based on several conditions mentioned in README file and the article (section 4.1)

eta in the code below is ϵ which is chosen to be 0.1. So Let's apply that below:

In [13]:
z = (mean - max(y)) / std
c1 = np.log(2 * (22/7)) / 2
c2 = np.log((22/7) / 2) / 2
eta=0.1

if (z > -1).any:     # the first condition for log_h 
    log_h = np.log(norm.pdf(z) + z * norm.cdf(z) + 1e-10)    # I added "1e-10" to avoid division by zero

elif (-1/np.sqrt(eta) < z).any() and (z <= -1).any():     # the second condition for log_h 
    log_h = -z**2/(2-c1) + log1mexp(np.log(sp.erfcx(-z/np.sqrt(2)) * abs(z)) + c2)

elif (z <= -1 / np.sqrt(eta)).any():      # the third condition for log_h 
    log_h = -z**2/(2-c1) + 2* np.log(abs(z))

acquisition_function = log_h + np.log(std)
acquisition_function

array([-34.88535105, -27.39690094, -26.72933864, ..., -26.52679065,
       -27.19570979,  -3.50463995])

# Obtaining the next Query:

In [14]:
idx_max = np.argmax(acquisition_function)
next_query = X_grid[idx_max]
print(np.round(next_query, decimals=6))

[1. 1. 1. 1.]
